In [1]:
from datetime import datetime, timedelta
import psycopg2
import pandas as pd
from datetime import datetime, timedelta
from bson import ObjectId
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from sshtunnel import SSHTunnelForwarder
from dbutils.pooled_db import PooledDB

ssh_host = 'api.trustyapp.io'
ssh_username = 'ubuntu'
ssh_pem_file = './ng-prod.pem'
database_host = '127.0.0.1'  # SSH 터널을 통해 localhost로 접근
database_name = 'trusty'
database_username = 'server'
database_password = 'ygFAhsyTMDzB21VdLTnoq7djxwjDNzuc9yPGukm73otXER8ihD'

server = SSHTunnelForwarder(
    (ssh_host, 22),  # SSH 서버 주소와 포트
    ssh_username=ssh_username,
    ssh_pkey=ssh_pem_file,  # Private key 파일 경로
    remote_bind_address=(database_host, 5432)  # 원격 MySQL 서버 주소와 포트
)

server.start()

conn = psycopg2.connect(
    host='127.0.0.1',  # localhost를 통해 SSH 터널로 접속
    user=database_username,
    password=database_password,
    database=database_name,
    port=server.local_bind_port  # sshtunnel이 자동으로 할당한 포트
)
cursor = conn.cursor()

In [ ]:
conn.rollback()

qry = """
select *
from bank_transaction bt 
left join bank_account ba on ba.id = bt."bankAccountId" 
where ba."userId" = 418
order by bt."transactionAt" asc,  bt.id asc
        """
cursor.execute(qry)
result = cursor.fetchall()

# DataFrame 생성
df = pd.DataFrame(result, columns=[desc[0] for desc in cursor.description])
df.to_csv('./418.csv', index=False, encoding='utf-8-sig')

# 결과 확인
print(df.head())

                         createdAt                        updatedAt     id  \
0 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62305   
1 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62304   
2 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62302   
3 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62303   
4 2024-11-22 22:17:48.846660+00:00 2024-11-22 22:17:48.846660+00:00  62301   

  serviceProvider             transactionId    type  amount  \
0            mono  6741030728ffa62e52afd7de  credit       9   
1            mono  6741030728ffa62e52afd7dd   debit     261   
2            mono  6741030728ffa62e52afd7db   debit  260000   
3            mono  6741030728ffa62e52afd7dc  credit  260000   
4            mono  6741030728ffa62e52afd7da   debit  250000   

              transactionAt       description  bankAccountId  ...  \
0 2023-12-01 01:35:32+00:00  OWealth Interest             98  ...   
1 2023-12-01 0